In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns


matplotlib.rcParams['pdf.fonttype'] = 42  # no type-3
matplotlib.rcParams['ps.fonttype'] = 42

num_total_event = 30  
rec_per_event = 15  # num. of recommendations per meal event in each run per patient.
all_patients = {1:'child#001', 2:'child#002', 3:'child#003', 4:'child#004', 5:'child#005',
                6:'child#006', 7:'child#007', 8:'child#008', 9:'child#009', 10: 'child#010',
                11:'adolescent#001', 12:'adolescent#002', 13:'adolescent#003', 14:'adolescent#004', 15:'adolescent#005',
                16:'adolescent#006', 17:'adolescent#007', 18:'adolescent#008', 19:'adolescent#009', 20:'adolescent#010',
                21:'adult#001', 22:'adult#002', 23:'adult#003', 24:'adult#004', 25:'adult#005',
                26:'adult#006', 27:'adult#007', 28:'adult#008', 29:'adult#009', 30:'adult#010'}



ins_ceiling = 40
ins_spacing = 0.1
ins_space = np.arange(-10, ins_ceiling + ins_spacing, ins_spacing)

noise_std = 5
global_tmi = 0
global_tmbg = 150

postbg_target = 112.5
bg_omin = 70
bg_omax = 180


def get_perf(data, key, low_lim=bg_omin, up_lim=bg_omax, target=postbg_target, c_print=True):
    '''
    Prints and returns the performance metrics.
    Perf.Metrics: Mean and Std. Dev of PP-BG. Hypo-, Hyper-, and Optimal Freqs. LBGI, HBGI, RI (risk indices)

    data (pandas.DataFrame): Data Frame with context, insulin recommendations, and corresponding BG values
    key (string): BG Measurements related to a key (e.g. bg_post_vol)
    low_lim, up_lim (float): low and high ends of the target range
    target (float): target postbg value
    '''
    bg_data = data.loc[:, key].to_numpy()

    mean = np.mean(bg_data)
    std = np.std(bg_data)
    hyper_freq = np.mean(bg_data > up_lim)
    hypo_freq = np.mean(bg_data < low_lim)
    optimal_freq = np.mean((bg_data <= up_lim) & (bg_data >= low_lim))

    fBG = 1.509 * (np.log(bg_data) ** 1.084 - 5.381)
    rbg = 10 * (fBG ** 2)
    rl = [bg if fBG[ind] < 0 else 0 for ind, bg in enumerate(rbg)]
    rh = [bg if fBG[ind] > 0 else 0 for ind, bg in enumerate(rbg)]
    LBGI = np.nan_to_num(np.mean(rl))
    HBGI = np.nan_to_num(np.mean(rh))
    RI = LBGI + HBGI

    if c_print:
        print('\n{}\n==================='.format(key))
        print('Hyper range: {:.5f}'.format(hyper_freq))
        print('Hypo range: {:.5f}'.format(hypo_freq))
        print('Optimum range: {:.5f}'.format(optimal_freq))
        print('PostBG: {:.5f} +- {:.5f}'.format(mean, std))
        print('LBGI: {:.5f}, HBGI: {:.5f}, RI: {:.5f}\n'.format(LBGI, HBGI, RI))

    return mean, std, hyper_freq, hypo_freq, optimal_freq, LBGI, HBGI, RI


def print_perf(data, p_data_dict, keys, setup=False, index=-1):
    '''
        setup (string or bool): setup name (e.g. Single Meal Event (SME)), if False, do not print.
        index (int): data name for which the performance will be printed (-1 for all data and individuals together, 0 for all data, otherwise the patient index.)
        data (pandas.DataFrame): DataFrame containing the data of all patients.
        p_data_dict (dictionary): Keys are patient names (e.g. adult#001). Values are pandas.DataFrame objects for corresponding patients.
    '''
    if setup is not False:
        print('Setup: {}\n*******************\n'.format(setup))

    if index == -1:
        print_perf(data=data, keys=keys, p_data_dict=p_data_dict, index=0)
        for i in range(len(p_data_dict)):
            print_perf(data=data, keys=keys, p_data_dict=p_data_dict, index=[*all_patients.keys()][i])
    elif index == 0:
        print('OVERALL PERFORMANCE\n-------------------')
        for key in keys:
            _ = get_perf(data=data, key=key)
    else:
        print(all_patients[index])
        for key in keys:
            _ = get_perf(data=p_data_dict[all_patients[index]], key=key)


sme_ppbg_path = 'SME/ppbg'
sme_ppbg_data = pd.read_csv('./{}/nc_sme.csv'.format(sme_ppbg_path))

sme_ppbg_tc_path = 'SME/ppbg_tc'
sme_ppbg_tc_data = pd.read_csv('./{}/tc_sme.csv'.format(sme_ppbg_tc_path))

pat_type = 'all'

if pat_type == 'adult':
    sme_ppbg_data = sme_ppbg_data.loc[:4499,:]
    sme_ppbg_tc_data = sme_ppbg_tc_data.loc[:4499,:]
    all_patients = {1:'adult#001', 2:'adult#002', 3:'adult#003', 4:'adult#004', 5:'adult#005',
                    6:'adult#006', 7:'adult#007', 8:'adult#008', 9:'adult#009', 10:'adult#010'}

elif pat_type == 'adolescent':
    sme_ppbg_data = sme_ppbg_data.loc[4500:8999,:] 
    sme_ppbg_tc_data = sme_ppbg_tc_data.loc[4500:8999,:]
    all_patients = {1:'adolescent#001', 2:'adolescent#002', 3:'adolescent#003', 4:'adolescent#004', 5:'adolescent#005',
                    6:'adolescent#006', 7:'adolescent#007', 8:'adolescent#008', 9:'adolescent#009', 10:'adolescent#010'}

elif pat_type == 'child':
    sme_ppbg_data = sme_ppbg_data.loc[9000:,:] 
    sme_ppbg_tc_data = sme_ppbg_tc_data.loc[9000:,:]
    all_patients = {1:'child#001', 2:'child#002', 3:'child#003', 4:'child#004', 5:'child#005',
                    6:'child#006', 7:'child#007', 8:'child#008', 9:'child#009', 10: 'child#010'}

num_patient = len(all_patients)

sme_ppbg_dict = {pt:pd.read_csv('./{}/{}_sme.csv'.format(sme_ppbg_path, pt)) for pt in [*all_patients.values()]}
sme_ppbg_tc_dict = {pt:pd.read_csv('./{}/{}_sme.csv'.format(sme_ppbg_tc_path, pt)) for pt in [*all_patients.values()]}

#  merge all sme results
sme_data = pd.concat([sme_ppbg_data, sme_ppbg_tc_data], axis=1)

mme_ppbg_path = 'MME/ppbg'
mme_ppbg_data = pd.read_csv('./{}/mme_ppbg.csv'.format(mme_ppbg_path))
mme_ppbg_dict = {pt:pd.read_csv('./{}/{}_mme.csv'.format(mme_ppbg_path, pt)) for pt in [*all_patients.values()]}

keys_sme = ['ESCADA', 'TACO', 'STS', 'TS', 'Calc.']
print_perf(data=sme_ppbg_data, p_data_dict=sme_ppbg_dict, setup='SME - Normal Calculator', index=0, keys=keys_sme) 

for key in keys_sme:
    hyper_arr = np.zeros(len(all_patients))
    hypo_arr = np.zeros(len(all_patients))
    hbgi_arr = np.zeros(len(all_patients))
    lbgi_arr = np.zeros(len(all_patients))
    for i in range(len(all_patients)):
        _, _, hyper_arr[i], hypo_arr[i], _, lbgi_arr[i], hbgi_arr[i], _ = get_perf(data=sme_ppbg_dict[all_patients[i+1]], key=key, c_print=False)
    
    print('\n{} - SME Normal Calc.\n{}'.format(key, '-'*40))
    print('Hyper Freq.: {:.5f} +- {:.5f}'.format(np.mean(hyper_arr), np.std(hyper_arr)))
    print('Hypo Freq.: {:.5f} +- {:.5f}'.format(np.mean(hypo_arr), np.std(hypo_arr)))
    print('HBGI: {:.5f} +- {:.5f}'.format(np.mean(hbgi_arr), np.std(hbgi_arr)))
    print('LBGI: {:.5f} +- {:.5f}'.format(np.mean(lbgi_arr), np.std(lbgi_arr)))

keys_sme_tc = ['ESCADA-TC', 'STS-TC', 'Tuned Calc.']
print_perf(data=sme_ppbg_tc_data, p_data_dict=sme_ppbg_tc_dict, setup='SME - Tuned Calculator', index=0, keys=keys_sme_tc) 

for key in keys_sme_tc:
    hyper_arr = np.zeros(len(all_patients))
    hypo_arr = np.zeros(len(all_patients))
    hbgi_arr = np.zeros(len(all_patients))
    lbgi_arr = np.zeros(len(all_patients))
    for i in range(len(all_patients)):
        _, _, hyper_arr[i], hypo_arr[i], _, lbgi_arr[i], hbgi_arr[i], _ = get_perf(data=sme_ppbg_tc_dict[all_patients[i+1]], key=key, c_print=False)
    
    print('\n{} - SME Tuned Calc.\n{}'.format(key, '-'*40))
    print('Hyper Freq.: {:.5f} +- {:.5f}'.format(np.mean(hyper_arr), np.std(hyper_arr)))
    print('Hypo Freq.: {:.5f} +- {:.5f}'.format(np.mean(hypo_arr), np.std(hypo_arr)))
    print('HBGI: {:.5f} +- {:.5f}'.format(np.mean(hbgi_arr), np.std(hbgi_arr)))
    print('LBGI: {:.5f} +- {:.5f}'.format(np.mean(lbgi_arr), np.std(lbgi_arr)))


all_patients = {1:'child#001', 2:'child#002', 3:'child#003', 4:'child#004', 5:'child#005',
                6:'child#006', 7:'child#007', 8:'child#008', 9:'child#009', 10: 'child#010',
                11:'adolescent#001', 12:'adolescent#002', 13:'adolescent#003', 14:'adolescent#004', 15:'adolescent#005',
                16:'adolescent#006', 17:'adolescent#007', 18:'adolescent#008', 19:'adolescent#009', 20:'adolescent#010',
                21:'adult#001', 22:'adult#002', 23:'adult#003', 24:'adult#004', 25:'adult#005',
                26:'adult#006', 27:'adult#007', 28:'adult#008', 29:'adult#009', 30:'adult#010'}

keys_mme = ['ESCADA', 'TACO', 'STS', 'TS', 'GP-UCB-1', 'GP-UCB-2', 'GP-UCB-3']
print_perf(data=mme_ppbg_data, p_data_dict=mme_ppbg_dict, setup='MME', index=2, keys=keys_mme) 

for key in keys_mme:
    hyper_arr = np.zeros(len(all_patients))
    hypo_arr = np.zeros(len(all_patients))
    hbgi_arr = np.zeros(len(all_patients))
    lbgi_arr = np.zeros(len(all_patients))
    for i in range(len(all_patients)):
        _, _, hyper_arr[i], hypo_arr[i], _, lbgi_arr[i], hbgi_arr[i], _ = get_perf(data=mme_ppbg_dict[all_patients[i+1]], key=key, c_print=False)
    
    print('\n{} - MME\n{}'.format(key, '-'*40))
    print('Hyper Freq.: {:.5f} +- {:.5f}'.format(np.mean(hyper_arr), np.std(hyper_arr)))
    print('Hypo Freq.: {:.5f} +- {:.5f}'.format(np.mean(hypo_arr), np.std(hypo_arr)))
    print('HBGI: {:.5f} +- {:.5f}'.format(np.mean(hbgi_arr), np.std(hbgi_arr)))
    print('LBGI: {:.5f} +- {:.5f}'.format(np.mean(lbgi_arr), np.std(lbgi_arr)))

In [ ]:
##### FIGURE 4

cps='tab10'
fs = (6.4, 4.8)

patient = ['adult#001', 'adult#006', 'adult#001']
meal_ind = [6, 12, 3]
color_ind = [0,6,2]

fig, ax = plt.subplots(figsize=fs)
sns.set_style("white")
ax.grid(False)
ax.set_yticks([bg_omin, postbg_target, bg_omax])

for k, i in enumerate(meal_ind):

    plot_data = pd.read_csv('./{}/{}_meal{}_sme.csv'.format(sme_ppbg_path, patient[k], i))
    pump_dose = plot_data.loc[0, 'ins_calc']
    recs_escada = np.array([*plot_data.loc[:, 'ins_escada']])
    bgs_escada = np.array([*plot_data.loc[:, 'ESCADA']])

    ax.scatter(recs_escada, bgs_escada, color=sns.color_palette(cps)[color_ind[k]], marker='d')
    ax.scatter(recs_escada[0], bgs_escada[0], color='black', marker='d')
    ax.plot(recs_escada, bgs_escada, ls='-', color=sns.color_palette(cps)[color_ind[k]])

ax.axhline(postbg_target, color=sns.color_palette()[2], alpha=1, ls='--')
ax.axhline(bg_omin, color=sns.color_palette()[3], alpha=1, ls='--')
ax.axhline(bg_omax, color=sns.color_palette()[3], alpha=1, ls='--')

l1 = mlines.Line2D([], [], color=sns.color_palette(cps)[color_ind[1]], ls='-', marker='d', label='ME 1')
l2 = mlines.Line2D([], [], color=sns.color_palette(cps)[color_ind[2]], ls='-', marker='d', label='ME 2')
l3 = mlines.Line2D([], [], color=sns.color_palette(cps)[color_ind[0]], ls='-', marker='d', label='ME 3')

l5 = mlines.Line2D([], [], color=sns.color_palette()[2], ls='--', label='Target BG')
l6 = mlines.Line2D([], [], color=sns.color_palette()[3], ls='--', label='Safety Bounds')

l7 = mlines.Line2D([], [], color='black', marker='d', ls='None', label='First recommendations')

leg1 = plt.legend(handles=[l5, l6], borderpad=0.4, loc=[0.52,0.68], facecolor='white', framealpha=1, prop={'size': 16})
leg2 = plt.legend(handles=[l1, l2, l3], borderpad=0.5, loc=[0.05,0.06], facecolor='white', framealpha=1, prop={'size': 16})
leg3 = plt.legend(handles=[l7], borderpad=0.5, loc=[0.36,0.08], facecolor='white', framealpha=1, prop={'size': 16})

ax.add_artist(leg1)
ax.add_artist(leg2)
ax.add_artist(leg3)

plt.xlabel('Insulin Dose (U)', fontsize=16)
plt.ylabel('Postprandial Blood Glucose (mg/dl)', fontsize=16)
plt.tick_params(labelsize=16)

plt.savefig('./figures/recovery.pdf', bbox_inches='tight')

In [ ]:
##### FIGURE 6

num_ev = num_total_event
color_ind = [2, 6, 0]

patient = ['adult#010', 'adult#001', 'child#003']
meal_indices = [13, 20, 7]
optimal_doses = [17.1, 24.8, 2.15]
sme_rec = {m: np.array(sme_ppbg_dict[patient[m]].loc[meal_indices[m]*rec_per_event:(meal_indices[m] + 1)*rec_per_event - 1, 'ins_escada']) for m in range(len(meal_indices))}
mme_rec = {m: np.array(mme_ppbg_dict[patient[m]].loc[[*range(meal_indices[m], meal_indices[m] + (rec_per_event)*num_ev, num_ev)], 'ins_escada']) for m in range(len(meal_indices))}

fig, ax = plt.subplots(figsize=fs)
ax.set_xticks(np.arange(1, rec_per_event + 1))
ax.grid(False)
sns.set_style("white")

for i in range(len(meal_indices)):
    labels = ['_nolegend_', '_nolegend_', '_nolegend_'] if i > 0 else ['SME', 'MME', 'Optimal']
    ax.plot(np.arange(1, rec_per_event + 1), sme_rec[i], color=sns.color_palette('{}'.format(cps))[color_ind[i]], alpha=1, label=labels[0], marker='d', linestyle='-')
    ax.plot(np.arange(1, rec_per_event + 1), mme_rec[i], color=sns.color_palette('{}'.format(cps))[color_ind[i]], alpha=1, label=labels[0], marker='x', markersize=8, linestyle='-')
    ax.axhline(optimal_doses[i], linestyle='--', color=sns.color_palette('{}'.format(cps))[color_ind[i]], alpha=1, label=labels[2])

l1 = mlines.Line2D([], [], color=sns.color_palette(cps)[1], marker='d', ls='-', label='SME')
l2 = mlines.Line2D([], [], color=sns.color_palette(cps)[1], marker='x', ls='-', markersize=8, label='MME')
l3 = mlines.Line2D([], [], color=sns.color_palette(cps)[1], marker='', ls='--', label='Opt.')

l4 = mlines.Line2D([], [], color=sns.color_palette(cps)[4], ls='-', label='ME 1')
l5 = mlines.Line2D([], [], color=sns.color_palette(cps)[2], ls='-', label='ME 2')
l6 = mlines.Line2D([], [], color=sns.color_palette(cps)[0], ls='-', label='ME 3')

leg1 = plt.legend(handles=[l1, l2, l3], borderpad=0.25, facecolor='white', framealpha=1, loc=(0.73, 0.15), prop={'size': 16})
leg2 = plt.legend(handles=[l4, l5, l6], borderpad=0.25, facecolor='white', framealpha=1, loc=(0.45, 0.15), prop={'size': 16})

ax.add_artist(leg1)
ax.add_artist(leg2)

plt.xlabel('Recommendation Number', fontsize=16)
plt.ylabel('Insulin Dose Recommendation (U)', fontsize=16)
plt.tick_params(labelsize=16)
plt.savefig('./figures/sme_mme_comparison.pdf', bbox_inches='tight')

In [ ]:
###### FIGURE 3

boxplot_data = sme_data.loc[:, {'TACO', 'ESCADA-TC', 'TS', 'STS-TC', 'Calc.', 'Tuned Calc.'}]
boxplot_data = boxplot_data.reindex(columns=['TACO', 'ESCADA-TC', 'TS', 'STS-TC', 'Calc.', 'Tuned Calc.'])

y_bins = np.array([40, 70, 112.5, 150, 180, 210, 240])

_, ax = plt.subplots(figsize=fs)
ax.set_yticks(y_bins)
ax.set(ylim=(40, 210))
plt.rcParams['axes.formatter.useoffset'] = False
sns.set_theme(style="whitegrid")

ax.axhline(postbg_target, color=sns.color_palette()[2], alpha=1, ls='--')
ax.axhline(bg_omin, color=sns.color_palette()[3], alpha=1, ls='--')
ax.axhline(bg_omax, color=sns.color_palette()[3], alpha=1, ls='--')

l3 = mlines.Line2D([], [], color=sns.color_palette()[2], ls='--', label='Target BG')
l4 = mlines.Line2D([], [], color=sns.color_palette()[3], ls='--', label='Safety Bounds')

plt.legend(handles=[l3, l4], borderpad=0.2, facecolor='white', framealpha=1, loc=(0.2, 0.7), prop={'size': 16})

my_pal = {"ESCADA-TC": sns.color_palette(cps)[1], "TACO": sns.color_palette(cps)[6], "TS":sns.color_palette(cps)[2], "STS-TC":sns.color_palette(cps)[0], "Calc.":sns.color_palette(cps)[3], "Tuned Calc.":sns.color_palette(cps)[8]}
ax = sns.boxplot(data=boxplot_data, fliersize=2, linewidth=1, saturation=1, palette=my_pal, showfliers=True, width=0.8)
ax.set_ylabel('Postprandial Blood Glucose (mg/dl)', fontsize=16)
ax.set_xlabel('Recommender', fontsize=16)
plt.tick_params(labelsize=13)

plt.savefig('./figures/sme_boxplot.pdf', bbox_inches='tight')

In [ ]:
###### FIGURE 5

run_count = num_total_event*rec_per_event
data = mme_ppbg_data

pt_regrets_escada = {pt: np.array([*map(lambda x:(np.abs(data.loc[i*run_count:i*run_count + x, 'ESCADA-N'] - postbg_target) ).sum(), [*range(run_count)])]) for i, pt in enumerate(all_patients.values())}
pt_regrets_taco = {pt: np.array([*map(lambda x:(np.abs(data.loc[i*run_count:i*run_count + x, 'TACO-N'] - postbg_target)).sum(), [*range(run_count)])]) for i, pt in enumerate(all_patients.values())}
pt_regrets_sts = {pt: np.array([*map(lambda x:(np.abs(data.loc[i*run_count:i*run_count + x, 'STS-N'] - postbg_target)).sum(), [*range(run_count)])]) for i, pt in enumerate(all_patients.values())}
pt_regrets_ts = {pt: np.array([*map(lambda x:(np.abs(data.loc[i*run_count:i*run_count + x, 'TS-N'] - postbg_target)).sum(), [*range(run_count)])]) for i, pt in enumerate(all_patients.values())}
pt_regrets_calc = {pt: np.array([*map(lambda x:(np.abs(data.loc[i*run_count:i*run_count + x, 'Calc.-N'] - postbg_target)).sum(), [*range(run_count)])]) for i, pt in enumerate(all_patients.values())}

regret_escada = sum(pt_regrets_escada.values())/len(pt_regrets_escada)
regret_taco = sum(pt_regrets_taco.values())/len(pt_regrets_taco)
regret_sts = sum(pt_regrets_sts.values())/len(pt_regrets_sts)
regret_ts = sum(pt_regrets_ts.values())/len(pt_regrets_ts)
regret_calc = sum(pt_regrets_calc.values())/len(pt_regrets_calc)

std_escada = np.std(np.dstack(tuple(pt_regrets_escada.values())), axis=2, ddof=1).reshape(run_count)
std_taco = np.std(np.dstack(tuple(pt_regrets_taco.values())), axis=2, ddof=1).reshape(run_count)
std_sts = np.std(np.dstack(tuple(pt_regrets_sts.values())), axis=2, ddof=1).reshape(run_count)
std_ts = np.std(np.dstack(tuple(pt_regrets_ts.values())), axis=2, ddof=1).reshape(run_count)
std_calc= np.std(np.dstack(tuple(pt_regrets_calc.values())), axis=2, ddof=1).reshape(run_count)

cps = 'tab10'

std_coeff = 0.25

t = np.arange(run_count) + 1
i = [*range(44, run_count, 45)]
sty = {
    'fillstyle': 'none',
    'linewidth': 1.0,
    'markevery': i}

plt.figure(figsize=fs)
sns.set_style("white")
plt.ticklabel_format(axis="y", style="sci", scilimits=(1,0))
colors_ord = [6,2,1,0,3]
alpha_val=0.2
lw=1
plt.grid(False)
plt.plot(t, regret_taco, color=sns.color_palette(cps)[colors_ord[0]], marker='*',  markersize=8, linestyle='--', **sty, label='TACO', LineWidth=lw)
plt.fill_between(t, regret_taco-std_coeff*std_taco, regret_taco+std_coeff*std_taco, color=sns.color_palette(cps)[colors_ord[0]], alpha=alpha_val)
plt.plot(t, regret_ts, color=sns.color_palette(cps)[colors_ord[1]], marker='s', linestyle='--', **sty, label='TS', LineWidth=lw)
plt.fill_between(t, regret_ts-std_coeff*std_ts, regret_ts+std_coeff*std_ts, color=sns.color_palette(cps)[colors_ord[1]], alpha=alpha_val)
plt.plot(t, regret_escada, color=sns.color_palette(cps)[colors_ord[2]], marker='x',  markersize=8, linestyle='--', **sty, label='ESCADA', LineWidth=lw)
plt.fill_between(t, regret_escada-std_coeff*std_escada, regret_escada+std_coeff*std_escada, color=sns.color_palette(cps)[colors_ord[2]], alpha=alpha_val)
plt.plot(t, regret_sts, color=sns.color_palette(cps)[colors_ord[3]], marker='d', linestyle='--', **sty, label='STS', LineWidth=lw)
plt.fill_between(t, regret_sts-std_coeff*std_sts, regret_sts+std_coeff*std_sts, color=sns.color_palette(cps)[colors_ord[3]], alpha=alpha_val)
plt.plot(t, regret_calc, color=sns.color_palette(cps)[colors_ord[4]], marker='o', linestyle='--', **sty, label='Calculator', LineWidth=lw)
plt.fill_between(t, regret_calc-std_coeff*std_calc, regret_calc+std_coeff*std_calc, color=sns.color_palette(cps)[colors_ord[4]], alpha=alpha_val)
plt.ylim(-5e1,3.4e3)

plt.legend(loc=[0.02,0.55], facecolor='white', framealpha=1, prop={'size': 15})
plt.xlabel('Rounds', fontsize=16)
plt.ylabel('Cumulative Regret', fontsize=16)
plt.tick_params(labelsize=16)

plt.savefig('./figures/mme_regret.pdf', bbox_inches='tight')

In [ ]:
##### FIGURE 7

colors_ord = [6,8,0,2,3,1]
cps = 'tab10'

my_pal = {"TACO": sns.color_palette(cps)[colors_ord[0]], "TS": sns.color_palette(cps)[colors_ord[3]], "GP-UCB-1": sns.color_palette(cps)[colors_ord[1]], "GP-UCB-2": sns.color_palette(cps)[colors_ord[2]],
         "GP-UCB-3": sns.color_palette(cps)[colors_ord[5]], "Calc.":sns.color_palette(cps)[colors_ord[4]]}

boxplot_data = mme_ppbg_data.loc[:, {'TACO', 'TS', 'GP-UCB-1', 'GP-UCB-2', 'GP-UCB-3', 'Calc.'}]
boxplot_data = boxplot_data.reindex(columns=['TACO', 'TS', 'GP-UCB-1', 'GP-UCB-2', 'GP-UCB-3', 'Calc.'])

y_bins = np.array([40, 70, 112.5, 150, 180, 210])

_, ax = plt.subplots(figsize=fs)
sns.set_theme(style="whitegrid")
ax.set_yticks(y_bins)
ax.set(ylim=(30, 210))
plt.rcParams['axes.formatter.useoffset'] = False

ax.axhline(postbg_target, color=sns.color_palette()[2], alpha=1, ls='--')
ax.axhline(bg_omin, color=sns.color_palette()[3], alpha=1, ls='--')
ax.axhline(bg_omax, color=sns.color_palette()[3], alpha=1, ls='--')

l3 = mlines.Line2D([], [], color=sns.color_palette()[2], ls='--', label='Target BG')
l4 = mlines.Line2D([], [], color=sns.color_palette()[3], ls='--', label='Safety Bounds')

plt.legend(handles=[l3, l4], borderpad=0.2, facecolor='white', framealpha=1, loc='upper center', prop={'size': 16})
ax = sns.boxplot(data=boxplot_data, fliersize=2, linewidth=1, saturation=1, palette=my_pal, showfliers=True, width=0.8)
ax.set_ylabel('Postprandial Blood Glucose (mg/dl)', fontsize=16)
ax.set_xlabel('Recommender', fontsize=16)
plt.tick_params(labelsize=12)

plt.savefig('./figures/mme_rebuttal_boxplot.pdf', bbox_inches='tight')

In [ ]:
##### FIGURE 8 and 9 

run_count = num_total_event*rec_per_event
data = mme_ppbg_data

pt_regrets_taco = {pt: np.array([*map(lambda x:(np.abs(data.loc[i*run_count:i*run_count + x, 'TACO-N'] - postbg_target)).sum(), [*range(run_count)])]) for i, pt in enumerate(all_patients.values())}
pt_regrets_ts = {pt: np.array([*map(lambda x:(np.abs(data.loc[i*run_count:i*run_count + x, 'TS-N'] - postbg_target)).sum(), [*range(run_count)])]) for i, pt in enumerate(all_patients.values())}
pt_regrets_gpucb1 = {pt: np.array([*map(lambda x:(np.abs(data.loc[i*run_count:i*run_count + x, 'GP-UCB-1-N'] - postbg_target) ).sum(), [*range(run_count)])]) for i, pt in enumerate(all_patients.values())}
pt_regrets_gpucb2 = {pt: np.array([*map(lambda x:(np.abs(data.loc[i*run_count:i*run_count + x, 'GP-UCB-2-N'] - postbg_target)).sum(), [*range(run_count)])]) for i, pt in enumerate(all_patients.values())}
pt_regrets_gpucb3 = {pt: np.array([*map(lambda x:(np.abs(data.loc[i*run_count:i*run_count + x, 'GP-UCB-3-N'] - postbg_target)).sum(), [*range(run_count)])]) for i, pt in enumerate(all_patients.values())}
pt_regrets_calc = {pt: np.array([*map(lambda x:(np.abs(data.loc[i*run_count:i*run_count + x, 'Calc.-N'] - postbg_target)).sum(), [*range(run_count)])]) for i, pt in enumerate(all_patients.values())}

regret_taco = sum(pt_regrets_taco.values())/len(pt_regrets_taco)
regret_ts = sum(pt_regrets_ts.values())/len(pt_regrets_ts)
regret_gpucb1 = sum(pt_regrets_gpucb1.values())/len(pt_regrets_gpucb1)
regret_gpucb2 = sum(pt_regrets_gpucb2.values())/len(pt_regrets_gpucb2)
regret_gpucb3 = sum(pt_regrets_gpucb3.values())/len(pt_regrets_gpucb3)
regret_calc = sum(pt_regrets_calc.values())/len(pt_regrets_calc)

std_taco = np.std(np.dstack(tuple(pt_regrets_taco.values())), axis=2, ddof=1).reshape(run_count)
std_ts = np.std(np.dstack(tuple(pt_regrets_ts.values())), axis=2, ddof=1).reshape(run_count)
std_gpucb1 = np.std(np.dstack(tuple(pt_regrets_gpucb1.values())), axis=2, ddof=1).reshape(run_count)
std_gpucb2 = np.std(np.dstack(tuple(pt_regrets_gpucb2.values())), axis=2, ddof=1).reshape(run_count)
std_gpucb3 = np.std(np.dstack(tuple(pt_regrets_gpucb2.values())), axis=2, ddof=1).reshape(run_count)
std_calc= np.std(np.dstack(tuple(pt_regrets_calc.values())), axis=2, ddof=1).reshape(run_count)

t = np.arange(run_count) + 1

sns.set_style("white")

plt.figure(figsize=fs)
plt.ticklabel_format(axis="y", style="sci", scilimits=(1,0))

plt.plot(t, regret_taco, color=sns.color_palette(cps)[colors_ord[0]], marker='*', markersize = 8, **sty, label='TACO')
plt.fill_between(t, regret_taco-std_coeff*std_taco, regret_taco+std_coeff*std_taco, color=sns.color_palette(cps)[colors_ord[0]], alpha=alpha_val)
plt.plot(t, regret_ts, color=sns.color_palette(cps)[colors_ord[3]], marker='o', markersize = 8, **sty, label='TS')
plt.fill_between(t, regret_ts-std_coeff*std_ts, regret_ts+std_coeff*std_ts, color=sns.color_palette(cps)[colors_ord[3]], alpha=alpha_val)
plt.plot(t, regret_gpucb1, color=sns.color_palette(cps)[colors_ord[1]], marker='s', **sty, label='GP-UCB-1')
plt.fill_between(t, regret_gpucb1-std_coeff*std_gpucb1, regret_gpucb1+std_coeff*std_gpucb1, color=sns.color_palette(cps)[colors_ord[1]], alpha=alpha_val)
plt.plot(t, regret_gpucb2, color=sns.color_palette(cps)[colors_ord[2]], marker='x', markersize = 8, **sty, label='GP-UCB-2')
plt.fill_between(t, regret_gpucb2-std_coeff*std_gpucb2, regret_gpucb2+std_coeff*std_gpucb2, color=sns.color_palette(cps)[colors_ord[2]], alpha=alpha_val)
plt.plot(t, regret_gpucb3, color=sns.color_palette(cps)[colors_ord[5]], marker='d', **sty, label='GP-UCB-3')
plt.fill_between(t, regret_gpucb3-std_coeff*std_gpucb3, regret_gpucb3+std_coeff*std_gpucb3, color=sns.color_palette(cps)[colors_ord[5]], alpha=alpha_val)
plt.plot(t, regret_calc, color=sns.color_palette(cps)[colors_ord[4]], LineStyle='--', **sty, label='Calc.')
plt.fill_between(t, regret_calc-std_coeff*std_calc, regret_calc+std_coeff*std_calc, color=sns.color_palette(cps)[colors_ord[4]], alpha=alpha_val)

plt.legend(loc='upper left', facecolor='white', framealpha=1, prop={'size': 16})
plt.xlabel('Rounds', fontsize=16)
plt.ylabel('Cumulative Regret', fontsize=16)
plt.tick_params(labelsize=16)

plt.savefig('./figures/mme_rebuttal_regret_1.pdf', bbox_inches='tight')

plt.figure(figsize=fs)
plt.ticklabel_format(axis="y", style="sci", scilimits=(1,0))
plt.plot(t, regret_taco, color=sns.color_palette(cps)[colors_ord[0]], marker='*', **sty, label='TACO')
plt.fill_between(t, regret_taco-std_coeff*std_taco, regret_taco  + std_coeff*std_taco, color=sns.color_palette(cps)[colors_ord[0]], alpha=alpha_val)
plt.plot(t, regret_ts, color=sns.color_palette(cps)[colors_ord[3]], marker='o', markersize = 8, **sty, label='TS')
plt.fill_between(t, regret_ts-std_coeff*std_ts, regret_ts+std_coeff*std_ts, color=sns.color_palette(cps)[colors_ord[3]], alpha=alpha_val)
plt.plot(t, regret_gpucb3, color=sns.color_palette(cps)[colors_ord[5]], marker='d', **sty, label='GP-UCB-3')
plt.fill_between(t, regret_gpucb3-std_coeff*std_gpucb3, regret_gpucb3+std_coeff*std_gpucb3, color=sns.color_palette(cps)[colors_ord[5]], alpha=alpha_val)
plt.legend(loc='upper left', facecolor='white', framealpha=1, prop={'size': 16})
plt.xlabel('Rounds', fontsize=16)
plt.ylabel('Cumulative Regret', fontsize=16)
plt.tick_params(labelsize=16)
plt.savefig('./figures/mme_rebuttal_regret_2.pdf', bbox_inches='tight')

In [ ]:
##### FIGURE 10

clinician_data = pd.read_csv('./clinician_comparison/test_res/test_res.csv')
boxplot_data = clinician_data.loc[:, {'meal', 'patient', 'bg_fasting', 'ESCADA', 'Clinicians', 'Calculator'}]
boxplot_data= boxplot_data.rename(columns={'Clinicians':'Clinician'})
boxplot_data = pd.melt(boxplot_data, id_vars=['patient', 'meal', 'bg_fasting'], value_vars=['ESCADA', 'Clinician', 'Calculator'], var_name='Method', value_name='Postprandial Blood Glucose (mg/dl)')
sns.set_theme(style="whitegrid")

y_bins = np.array([40, 70, 100, 112.5, 150, 180, 210, 240])

_, ax = plt.subplots(figsize=fs)
ax.set_yticks(y_bins)
ax.set(ylim=(30, 260))
plt.rcParams['axes.formatter.useoffset'] = False

ax.axhline(postbg_target, color=sns.color_palette()[2], alpha=1, ls='--')
ax.axhline(bg_omin, color=sns.color_palette()[3], alpha=1, ls='--')
ax.axhline(bg_omax, color=sns.color_palette()[3], alpha=1, ls='--')

l3 = mlines.Line2D([], [], color=sns.color_palette()[2], ls='--', label='Target BG')
l4 = mlines.Line2D([], [], color=sns.color_palette()[3], ls='--', label='Safety Bounds')

plt.legend(handles=[l3, l4], borderpad=0.2, loc=[0.45,0.008], facecolor='white', framealpha=1, prop={'size': 16})

my_pal = {"ESCADA": sns.color_palette(cps)[2], "Clinician": sns.color_palette(cps)[0], "Calculator":sns.color_palette(cps)[6]}
ax = sns.boxplot(x="Method", y="Postprandial Blood Glucose (mg/dl)", data=boxplot_data, fliersize=2, linewidth=2, saturation=1, palette=my_pal, showfliers=True)
ax.set_xlabel('Recommender', fontsize=16)
ax.set_ylabel('Postprandial Blood Glucose (mg/dl)', fontsize=16)
ax.tick_params(labelsize=16)

plt.savefig('./figures/doc_comp.pdf', bbox_inches='tight')
plt.show()

In [ ]:
##### FIGURE 11

boxplot_data = sme_data.loc[:, {'TACO', 'ESCADA-TC', 'TS', 'STS-TC', 'Calc.', 'Tuned Calc.'}]
boxplot_data = boxplot_data.reindex(columns=['TACO', 'ESCADA-TC', 'TS', 'STS-TC', 'Calc.', 'Tuned Calc.'])
boxplot_data.loc[:4499, 'Patient subgroup'], boxplot_data.loc[4500:8999, 'Patient subgroup'], boxplot_data.loc[9000:, 'Patient subgroup'] = 'Adult', 'Adolescent', 'Child'
boxplot_data_v1 = boxplot_data.melt(id_vars=['Patient subgroup'], var_name="Algorithm", value_name="PPBG")


y_bins = np.array([40, 70, 112.5, 150, 180, 210, 240])

_, ax = plt.subplots(figsize=(12.8,6.4))
ax.set_yticks(y_bins)
ax.set(ylim=(40, 210))
plt.rcParams['axes.formatter.useoffset'] = False
sns.set_theme(style="whitegrid")

ax.axhline(postbg_target, color=sns.color_palette()[2], alpha=1, linewidth=2, ls='--')
ax.axhline(bg_omin, color=sns.color_palette()[3], alpha=1, linewidth=2, ls='-')
ax.axhline(bg_omax, color=sns.color_palette()[3], alpha=1, linewidth=2, ls='-')

l3 = mlines.Line2D([], [], color=sns.color_palette()[2], ls='--', linewidth=2, label='Target BG')
l4 = mlines.Line2D([], [], color=sns.color_palette()[3], ls='-', linewidth=2, label='Safety Bounds')

leg1 = plt.legend(handles=[l3, l4], borderpad=0.25, fancybox=True, facecolor='white', framealpha=0.2, loc=(0.73, 0.01), prop={'size': 16})

ax = sns.boxplot(x="Algorithm", y="PPBG", hue="Patient subgroup", palette=sns.color_palette('colorblind'), data=boxplot_data_v1, fliersize=1, linewidth=1, saturation=1, showfliers=True, width=0.8)
ax.set_ylabel('Postprandial Blood Glucose (mg/dl)', fontsize=16)
ax.set_xlabel('Recommender', fontsize=16)
ax.tick_params(labelsize=16)
plt.legend(fontsize=16, loc=(0.8, 0.81), fancybox=True, framealpha=0.2)
ax.add_artist(leg1)

plt.savefig('./figures/rebut_bgp1.pdf', bbox_inches='tight')

boxplot_data = sme_data.loc[:, {'ins_taco', 'ins_escada_tc', 'ins_ts', 'ins_sts_tc', 'ins_calc', 'ins_calc_tc'}]
boxplot_data = boxplot_data.reindex(columns=['ins_taco', 'ins_escada_tc', 'ins_ts', 'ins_sts_tc', 'ins_calc', 'ins_calc_tc'])
boxplot_data = boxplot_data.rename(columns={'ins_taco': 'TACO', 'ins_escada_tc': 'ESCADA-TC', 'ins_ts': 'TS', 'ins_sts_tc': 'STS-TC', 'ins_calc': 'Calc.', 'ins_calc_tc': 'Tuned Calc.'})
boxplot_data.loc[:4499, 'Patient subgroup'], boxplot_data.loc[4500:8999, 'Patient subgroup'], boxplot_data.loc[9000:, 'Patient subgroup'] = 'Adult', 'Adolescent', 'Child'
boxplot_data_v1 = boxplot_data.melt(id_vars=['Patient subgroup'], var_name="Algorithm", value_name="Insulin")


y_bins = np.array([0, 10, 20, 30])

_, ax = plt.subplots(figsize=(12.8,6.4))
ax.set_yticks(y_bins)
ax.set(ylim=(0, 31))
plt.rcParams['axes.formatter.useoffset'] = False
sns.set_theme(style="whitegrid")

ax = sns.boxplot(x="Algorithm", y="Insulin", hue="Patient subgroup", palette=sns.color_palette('colorblind'), data=boxplot_data_v1, fliersize=1, linewidth=1, saturation=1, showfliers=True, width=0.8)
ax.set_ylabel('Insulin (U)', fontsize=16)
ax.set_xlabel('Recommender', fontsize=16)
ax.tick_params(labelsize=16)
plt.legend(fontsize=16, loc=(0.565, 0.78), fancybox=True, framealpha=0.2)

plt.savefig('./figures/rebut_ip1.pdf', bbox_inches='tight')

sme_ppbg_path_dt = 'SME/ppbg_dt'
sme_ppbg_data_dt = pd.read_csv('./{}/nc_sme.csv'.format(sme_ppbg_path_dt))

sme_ppbg_tc_path_dt = 'SME/ppbg_tc_dt'
sme_ppbg_tc_data_dt = pd.read_csv('./{}/tc_sme.csv'.format(sme_ppbg_tc_path_dt))

pat_type = 'all'

if pat_type == 'adult':
    sme_ppbg_data_dt = sme_ppbg_data_dt.loc[:4499,:]
    sme_ppbg_tc_data_dt = sme_ppbg_tc_data_dt.loc[:4499,:]
    all_patients = {1:'adult#001', 2:'adult#002', 3:'adult#003', 4:'adult#004', 5:'adult#005',
                    6:'adult#006', 7:'adult#007', 8:'adult#008', 9:'adult#009', 10:'adult#010'}

elif pat_type == 'adolescent':
    sme_ppbg_data_dt = sme_ppbg_data_dt.loc[4500:8999,:] 
    sme_ppbg_tc_data_dt = sme_ppbg_tc_data_dt.loc[4500:8999,:]
    all_patients = {1:'adolescent#001', 2:'adolescent#002', 3:'adolescent#003', 4:'adolescent#004', 5:'adolescent#005',
                    6:'adolescent#006', 7:'adolescent#007', 8:'adolescent#008', 9:'adolescent#009', 10:'adolescent#010'}

elif pat_type == 'child':
    sme_ppbg_data_dt = sme_ppbg_data_dt.loc[9000:,:] 
    sme_ppbg_tc_data_dt = sme_ppbg_tc_data_dt.loc[9000:,:]
    all_patients = {1:'child#001', 2:'child#002', 3:'child#003', 4:'child#004', 5:'child#005',
                    6:'child#006', 7:'child#007', 8:'child#008', 9:'child#009', 10: 'child#010'}

num_patient = len(all_patients)

sme_ppbg_dict_dt = {pt:pd.read_csv('./{}/{}_sme.csv'.format(sme_ppbg_path_dt, pt)) for pt in [*all_patients.values()]}
sme_ppbg_tc_dict_dt = {pt:pd.read_csv('./{}/{}_sme.csv'.format(sme_ppbg_tc_path_dt, pt)) for pt in [*all_patients.values()]}

#  merge all sme results
sme_data_dt = pd.concat([sme_ppbg_data_dt, sme_ppbg_tc_data_dt], axis=1)


keys_sme = ['ESCADA', 'TACO', 'STS', 'TS', 'Calc.']
print_perf(data=sme_ppbg_data_dt, p_data_dict=sme_ppbg_dict_dt, setup='SME - Normal Calculator', index=0, keys=keys_sme) 

for key in keys_sme:
    hyper_arr = np.zeros(len(all_patients))
    hypo_arr = np.zeros(len(all_patients))
    hbgi_arr = np.zeros(len(all_patients))
    lbgi_arr = np.zeros(len(all_patients))
    for i in range(len(all_patients)):
        _, _, hyper_arr[i], hypo_arr[i], _, lbgi_arr[i], hbgi_arr[i], _ = get_perf(data=sme_ppbg_dict_dt[all_patients[i+1]], key=key, c_print=False)
    
    print('\n{} - SME Normal Calc.\n{}'.format(key, '-'*40))
    print('Hyper Freq.: {:.5f} +- {:.5f}'.format(np.mean(hyper_arr), np.std(hyper_arr)))
    print('Hypo Freq.: {:.5f} +- {:.5f}'.format(np.mean(hypo_arr), np.std(hypo_arr)))
    print('HBGI: {:.5f} +- {:.5f}'.format(np.mean(hbgi_arr), np.std(hbgi_arr)))
    print('LBGI: {:.5f} +- {:.5f}'.format(np.mean(lbgi_arr), np.std(lbgi_arr)))

keys_sme_tc = ['ESCADA-TC', 'STS-TC', 'Tuned Calc.']
print_perf(data=sme_ppbg_tc_data_dt, p_data_dict=sme_ppbg_tc_dict_dt, setup='SME - Tuned Calculator', index=0, keys=keys_sme_tc) 

for key in keys_sme_tc:
    hyper_arr = np.zeros(len(all_patients))
    hypo_arr = np.zeros(len(all_patients))
    hbgi_arr = np.zeros(len(all_patients))
    lbgi_arr = np.zeros(len(all_patients))
    for i in range(len(all_patients)):
        _, _, hyper_arr[i], hypo_arr[i], _, lbgi_arr[i], hbgi_arr[i], _ = get_perf(data=sme_ppbg_tc_dict_dt[all_patients[i+1]], key=key, c_print=False)
    
    print('\n{} - SME Tuned Calc.\n{}'.format(key, '-'*40))
    print('Hyper Freq.: {:.5f} +- {:.5f}'.format(np.mean(hyper_arr), np.std(hyper_arr)))
    print('Hypo Freq.: {:.5f} +- {:.5f}'.format(np.mean(hypo_arr), np.std(hypo_arr)))
    print('HBGI: {:.5f} +- {:.5f}'.format(np.mean(hbgi_arr), np.std(hbgi_arr)))
    print('LBGI: {:.5f} +- {:.5f}'.format(np.mean(lbgi_arr), np.std(lbgi_arr)))

In [ ]:
##### FIGURE 12


boxplot_data = sme_data_dt.loc[:, {'TACO', 'ESCADA-TC', 'TS', 'STS-TC', 'Calc.', 'Tuned Calc.'}]
boxplot_data = boxplot_data.reindex(columns=['TACO', 'ESCADA-TC', 'TS', 'STS-TC', 'Calc.', 'Tuned Calc.'])
boxplot_data.loc[:4499, 'Patient subgroup'], boxplot_data.loc[4500:8999, 'Patient subgroup'], boxplot_data.loc[9000:, 'Patient subgroup'] = 'Adult', 'Adolescent', 'Child'
boxplot_data_v1 = boxplot_data.melt(id_vars=['Patient subgroup'], var_name="Algorithm", value_name="PPBG")


y_bins = np.array([40, 70, 110, 125, 140, 180, 210, 240])

_, ax = plt.subplots(figsize=(12.8,6.4))
ax.set_yticks(y_bins)
ax.set(ylim=(40, 210))
plt.rcParams['axes.formatter.useoffset'] = False
sns.set_theme(style="whitegrid")

ax.axhline(110, color=sns.color_palette()[1], alpha=1, linewidth=2, ls='--')
ax.axhline(125, color=sns.color_palette()[2], alpha=1, linewidth=2, ls='--')
ax.axhline(140, color=sns.color_palette()[0], alpha=1, linewidth=2, ls='--')
ax.axhline(bg_omin, color=sns.color_palette()[3], alpha=1, linewidth=2, ls='-')
ax.axhline(bg_omax, color=sns.color_palette()[3], alpha=1, linewidth=2, ls='-')

l1 = mlines.Line2D([], [], color=sns.color_palette()[1], ls='--', linewidth=2, label='Adolescent Target BG')
l2 = mlines.Line2D([], [], color=sns.color_palette()[2], ls='--', linewidth=2, label='Child Target BG')
l3 = mlines.Line2D([], [], color=sns.color_palette()[0], ls='--', linewidth=2, label='Adult Target BG')
l4 = mlines.Line2D([], [], color=sns.color_palette()[3], ls='-', linewidth=2, label='Safety Bounds')

leg1 = plt.legend(handles=[l1, l2, l3, l4], borderpad=0.25, fancybox=True, facecolor='white', framealpha=0.2, loc=(0.64, -0.015), prop={'size': 16})

ax = sns.boxplot(x="Algorithm", y="PPBG", hue="Patient subgroup", palette=sns.color_palette('colorblind'), data=boxplot_data_v1, fliersize=1, linewidth=1, saturation=1, showfliers=True, width=0.8)
ax.set_ylabel('Postprandial Blood Glucose (mg/dl)', fontsize=16)
ax.set_xlabel('Recommender', fontsize=16)
ax.tick_params(labelsize=16)
plt.legend(fontsize=16, loc=(0.81, 0.81), fancybox=True, framealpha=0.2)
ax.add_artist(leg1)

plt.savefig('./figures/rebut_bgp2.pdf', bbox_inches='tight')

boxplot_data = sme_data_dt.loc[:, {'ins_taco', 'ins_escada_tc', 'ins_ts', 'ins_sts_tc', 'ins_calc', 'ins_calc_tc'}]
boxplot_data = boxplot_data.reindex(columns=['ins_taco', 'ins_escada_tc', 'ins_ts', 'ins_sts_tc', 'ins_calc', 'ins_calc_tc'])
boxplot_data = boxplot_data.rename(columns={'ins_taco': 'TACO', 'ins_escada_tc': 'ESCADA-TC', 'ins_ts': 'TS', 'ins_sts_tc': 'STS-TC', 'ins_calc': 'Calc.', 'ins_calc_tc': 'Tuned Calc.'})
boxplot_data.loc[:4499, 'Patient subgroup'], boxplot_data.loc[4500:8999, 'Patient subgroup'], boxplot_data.loc[9000:, 'Patient subgroup'] = 'Adult', 'Adolescent', 'Child'
boxplot_data_v1 = boxplot_data.melt(id_vars=['Patient subgroup'], var_name="Algorithm", value_name="Insulin")


y_bins = np.array([0, 10, 20, 30])

_, ax = plt.subplots(figsize=(12.8,6.4))
ax.set_yticks(y_bins)
ax.set(ylim=(0, 31))
plt.rcParams['axes.formatter.useoffset'] = False
sns.set_theme(style="whitegrid")

ax = sns.boxplot(x="Algorithm", y="Insulin", hue="Patient subgroup", palette=sns.color_palette('colorblind'), data=boxplot_data_v1, fliersize=1, linewidth=1, saturation=1, showfliers=True, width=0.8)
ax.set_ylabel('Insulin (U)', fontsize=16)
ax.set_xlabel('Recommender', fontsize=16)
ax.tick_params(labelsize=16)
plt.legend(fontsize=16, loc=(0.6, 0.78), fancybox=True, framealpha=0.2)

plt.savefig('./figures/rebut_ip2.pdf', bbox_inches='tight')